# 'This is Jeopardy!' Data Analysis

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

## Project Requirements
### Read CSV

In [2]:
df = pd.read_csv('jeopardy.csv')

df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,"No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves",Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,"The city of Yuma in this state has a record average of 4,055 hours of sunshine each year",Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", this company served its billionth burger",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States",John Adams


### Column names
The column names have a space at the start which is resolved as follows:

In [3]:
df.columns = df.columns.str.lstrip()

To allow dot notation, I also want all lowercase & underscores instead of spaces 

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

### Filter questions for given word(s)

In [5]:
def word_filter(df, words):
    return df.loc[df.question.apply(lambda x:
                                    all(word.lower() in x.lower()
                                        for word in words))]

word_filter(df, ['England', 'king'])

,show_number,air_date,round,category,value,question,answer
4953,3003,1997-09-24,Double Jeopardy!,"""PH""UN WORDS",$200,"Both England's King George V & FDR put their stamp of approval on this ""King of Hobbies""",Philately (stamp collecting)
6337,3517,1999-12-14,Double Jeopardy!,Y1K,$800,"In retaliation for Viking raids, this ""Unready"" king of England attacks Norse areas of the Isle of Man",Ethelred
9191,3907,2001-09-04,Double Jeopardy!,WON THE BATTLE,$800,This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt,Henry V
11710,2903,1997-03-26,Double Jeopardy!,BRITISH MONARCHS,$600,"This Scotsman, the first Stuart king of England, was called ""The Wisest Fool in Christendom""",James I
13454,4726,2005-03-07,Jeopardy!,A NUMBER FROM 1 TO 10,$1000,It's the number that followed the last king of England named William,4
...,...,...,...,...,...,...,...
208295,4621,2004-10-11,Jeopardy!,THE VIKINGS,$600,In 1066 this great-great grandson of Rollo made what some call the last Viking invasion of England,William the Conqueror
208742,4863,2005-11-02,Double Jeopardy!,BEFORE & AFTER,"$3,000",Dutch-born king who ruled England jointly with Mary II & is a tasty New Zealand fish,William of Orange roughy
213870,5856,2010-02-15,Double Jeopardy!,URANUS,$1600,In 1781 William Herschel discovered Uranus & initially named it after this king of England,George III
216021,1881,1992-11-09,Double Jeopardy!,HISTORIC NAMES,$1000,"His nickname was ""Bertie"", but he used this name & number when he became king of England in 1901",Edward VII


### Convert `value` column to `float` type

In [6]:
df.dtypes

show_number     int64
air_date       object
round          object
category       object
value          object
question       object
answer         object
dtype: object

In [7]:
df.value = df.value.str.extract('(\d+)').astype(float)

### Use `word_filter` to measure "difficulty" of given topics
Using the word `'King'` as an example, calculating the mean value of question:

In [8]:
def mean_price(df, words):
    return word_filter(df, words).value.mean()

mean_price(df, ['King'])

691.5493886522873

### Count of unique answers
Using the word `'King'` as an example, return count of unique answers to all questions in dataset:

In [9]:
def unique_answers_count(df):
    return df.answer.value_counts()

unique_answers_count(word_filter(df, ['King']))

Henry VIII          55
Solomon             35
Richard III         33
Louis XIV           31
David               30
                    ..
hyena                1
Viceroy              1
"the Unready"        1
a copper shopper     1
the stone            1
Name: answer, Length: 5268, dtype: int64

## Exploratory Data Analysis

### As time goes by...
Exploring how, grouped by decades, various parts of the show change

#### Preliminary cleaning

##### Converting `air_date` to `decade`

In [10]:
df.air_date = pd.to_datetime(df.air_date)

df['decade'] = ((df.air_date.dt.year)//10)*10

df.decade = df.decade.map(str) + 's'

print(df.decade)

0         2000s
1         2000s
2         2000s
3         2000s
4         2000s
          ...  
216925    2000s
216926    2000s
216927    2000s
216928    2000s
216929    2000s
Name: decade, Length: 216930, dtype: object


#### Most common categories

In [11]:
category_decade = df.groupby(['decade', 'category'])\
                    .category\
                    .count()\
                    .reset_index(name='count')\
                    .pivot(columns='decade', 
                           index='category', 
                           values='count')

Replacing all NaN (null values) as zeroes, and converting count type from `float` to `int`

In [17]:
category_decade = category_decade.fillna(0).astype(int)

category_decade.head()

decade,1980s,1990s,2000s,2010s
category,,,,
A JIM CARREY FILM FESTIVAL,0,0,5,0
"""!""",0,0,5,0
"""-ARES""",0,0,0,5
"""-ICIAN"" EXPEDITION",0,0,5,0
"""...OD"" WORDS",0,0,5,0


##### Top 10 categories in each decade

`for` loop to:
1. sort all columns by descending,
2. then, extracting the top 10
3. then, converting counts to integers (`int`)
4. appending the DataFrames to a list in order to concatenate

In [13]:
top10_cats_dfs = []

for column in category_decade:
    top10_cats = category_decade[column]\
                    .sort_values(ascending=False)\
                    .head(10)\
                    .astype(int)\
                    .reset_index()
    top10_cats_dfs.append(top10_cats)

top10_cats_decade = pd.concat(top10_cats_dfs, axis=1)

top10_cats_decade

,category,1980s,category,1990s,category,2000s,category,2010s
0,ANIMALS,59,LITERATURE,276,BEFORE & AFTER,391,POP CULTURE,40
1,SCIENCE,57,SPORTS,236,SCIENCE,203,STUPID ANSWERS,35
2,SPORTS,56,SCIENCE,234,STUPID ANSWERS,195,CLASSICAL MUSIC,35
3,U.S. HISTORY,51,AMERICAN HISTORY,229,AMERICANA,173,ABBREV.,35
4,FOOD,51,WORLD HISTORY,229,WORD ORIGINS,170,COLLEGES & UNIVERSITIES,30
5,GEOGRAPHY,48,WORLD GEOGRAPHY,224,BODIES OF WATER,165,THE HUMAN BODY,29
6,LITERATURE,48,HISTORY,220,POTPOURRI,164,WORD ORIGINS,28
7,RELIGION,48,BUSINESS & INDUSTRY,205,LITERATURE,162,OPERA,27
8,TRANSPORTATION,47,TELEVISION,201,RHYME TIME,159,ART & ARTISTS,26
9,QUOTES,45,COLLEGES & UNIVERSITIES,183,AMERICAN HISTORY,153,NONFICTION,25


Adding a `rank` column, and renaming columns:

In [14]:
top10_cats_decade.insert(0, 'rank', range(1, 11))

top10_cats_decade.columns = ['rank', 
                             '1980s_cat', '1980s_cat_count', 
                             '1990s_cat', '1990s_cat_count', 
                             '2000s_cat', '2000s_cat_count', 
                             '2010s_cat', '2010s_cat_count']

top10_cats_decade

,rank,1980s_cat,1980s_cat_count,1990s_cat,1990s_cat_count,2000s_cat,2000s_cat_count,2010s_cat,2010s_cat_count
0,1,ANIMALS,59,LITERATURE,276,BEFORE & AFTER,391,POP CULTURE,40
1,2,SCIENCE,57,SPORTS,236,SCIENCE,203,STUPID ANSWERS,35
2,3,SPORTS,56,SCIENCE,234,STUPID ANSWERS,195,CLASSICAL MUSIC,35
3,4,U.S. HISTORY,51,AMERICAN HISTORY,229,AMERICANA,173,ABBREV.,35
4,5,FOOD,51,WORLD HISTORY,229,WORD ORIGINS,170,COLLEGES & UNIVERSITIES,30
5,6,GEOGRAPHY,48,WORLD GEOGRAPHY,224,BODIES OF WATER,165,THE HUMAN BODY,29
6,7,LITERATURE,48,HISTORY,220,POTPOURRI,164,WORD ORIGINS,28
7,8,RELIGION,48,BUSINESS & INDUSTRY,205,LITERATURE,162,OPERA,27
8,9,TRANSPORTATION,47,TELEVISION,201,RHYME TIME,159,ART & ARTISTS,26
9,10,QUOTES,45,COLLEGES & UNIVERSITIES,183,AMERICAN HISTORY,153,NONFICTION,25


#### Most common answers
Similarly as finding the most common `category` in each decade, finding the top 10 `answer` in each decade:

##### Counting distinct answers by decade

In [15]:
answer_decade = df.groupby(['decade', 'answer'])\
                    .category\
                    .count()\
                    .reset_index(name='count')\
                    .pivot(columns='decade', 
                           index='answer', 
                           values='count')

answer_decade = answer_decade.fillna(0).astype(int)

answer_decade

decade,1980s,1990s,2000s,2010s
answer,,,,
Hamlet,0,1,0,0
Les Miserables,0,0,0,1
Nosferatu,0,1,0,0
She Loves You,0,1,0,0
Sleepless in Seattle,0,0,1,0
...,...,...,...,...
étoufée,0,0,1,1
études,0,0,1,0
été,0,0,0,1


##### Finding top 10 answers for each decade

In [16]:
top10_ans_dfs = []

for column in answer_decade:
    top10_ans = answer_decade[column]\
                .sort_values(ascending=False)\
                .head(10)\
                .astype(int)\
                .reset_index()
    top10_ans_dfs.append(top10_ans)

top10_ans_decade = pd.concat(top10_ans_dfs, axis=1)

top10_ans_decade.insert(0, 'rank', range(1, 11))

top10_ans_decade.columns = ['rank', 
                             '1980s_ans', '1980s_ans_count', 
                             '1990s_ans', '1990s_ans_count', 
                             '2000s_ans', '2000s_ans_count', 
                             '2010s_ans', '2010s_ans_count']

top10_ans_decade

,rank,1980s_ans,1980s_ans_count,1990s_ans,1990s_ans_count,2000s_ans,2000s_ans_count,2010s_ans,2010s_ans_count
0,1,France,11,Australia,66,Australia,126,China,31
1,2,3,10,Spain,60,China,121,Chicago,29
2,3,George Washington,10,China,57,Chicago,116,New York,25
3,4,Japan,9,France,55,Japan,110,India,23
4,5,Abraham Lincoln,9,Japan,55,California,108,California,23
5,6,India,8,Alaska,48,India,106,Mexico,23
6,7,Ronald Reagan,8,Hawaii,48,France,104,France,23
7,8,Egypt,8,India,48,Canada,103,Russia,23
8,9,Alaska,8,Paris,48,Italy,102,Washington,22
9,10,New York,8,Canada,47,Mexico,90,Florida,22
